In [ ]:
# -== ARQUIVO COM FUNCOES CRIADAS PARA A LEITURA MANIPULACAO DE DADOS =====================

In [1]:
# ========================= IMPORTANDO PACOTES================================================================== 

import os
import sys
import time as t
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4 as nc
import datetime as dt

from osgeo import osr
from osgeo import gdal
from netCDF4 import Dataset

# ============================================================================

In [ ]:
# ======================= FUNCAO RETORNA LISTA DE ARQUIVOS -=======================

def retorna_lista(caminho_do_arquivo):
    lista = []
    for nome_arquivo in os.listdir(caminho_do_arquivo):
        aux = os.path.join(caminho_do_arquivo, nome_arquivo)
        lista.append(aux)
    lista = sorted(lista)
    return lista 
#------------------------------------------------------------------
# -- DESCRICAO ---
#  chamando a funcao enviando caminho do diretorio e a funcao retorna um array com a lista ordenada dos arquivos


In [1]:
#================================ Funcao separa por altura do csv do INPE ========================================
#
# CRIANDO FUNÇÃO PARA SERPARAR POR PRESSAO  versao 1.0
# def separando_dados_altura(arquivo, pressao1, pressao2):
#     arquivo = pd.DataFrame(arquivo)
#     arquivo = arquivo.astype(float)
    
#     #aux = arquivo[np.logical_or(arquivo['press'] > int(altura1) , arquivo['press'] < int(altura2) )]
#     aux =  arquivo.loc[ arquivo['press'] <= int(pressao2)]
#     aux = aux.loc[arquivo['press'] >= int(pressao1)]
#     return aux

# =============== VERSAO ATUALIZADA 19-09-23 =============================================================
def separando_dados_altura(arquivo, valor1, valor2, VAR):
    arquivo = pd.DataFrame(arquivo)
    arquivo = arquivo.astype(float)
        
#     aux =  arquivo.loc[ arquivo[VAR] <= int(valor2)]
#     aux = aux.loc[aux[VAR] >= int(valor1)]
    
    aux = arquivo.loc[(arquivo[VAR] >= float(valor1)) &
                      (arquivo[VAR] <= float(valor2))
                     ]

    return aux


#------------------------------------------------------------------
# -- DESCRICAO ---
#  chame a funcao  enviando um data frame e ele retorna a coluna  somente os dados com a pressao desejada 
#  Tem que manipular corretamente 

#==============================FUNCIONA EXCLUSIVAMENTE PARA OS ARQUIVOS DE CSV INPE===============================

In [ ]:
#================================ Funcao SEPARA DETERMINADA LAT LON ========================================
## QUERO DETERMINADA  LAT E LON
def grade_lat_lon(arquivo, lat_sup, lat_inf, lon_esq,lon_dir):  
    arquivo = pd.DataFrame(arquivo)
    arquivo = arquivo.astype(float)
    aux = arquivo.loc[(arquivo['lat'] <= float(lat_sup)) & 
                  (arquivo['lat'] >= float(lat_inf)) & 
                  (arquivo['lon'] >= float(lon_esq)) &
                  (arquivo['lon'] <= float(lon_dir))
                     ] 
    return aux
#==================================================================================================================
# ========= EXPLICAÇÕES ============
# FUNCAO PARA PEGAR DETERMINADA LAT E LON NO ARQUIVO CSV INPE 
# ONDE  LAT_SUP É A LAT MAIS PROXIMA AO EIXO DO EQUADOR
# LAT_IN É A MAIS PROXIMA DO ARTICO
# LONG_ESQ É A LAT DE MAIOR VALOR NEGATIVO
#LONG_DIR É A DE MENOR VALOR , MAIS PROXIMO AO MERIDIANO DE GREENWICH
#==================================================================================================================
#==============================FUNCIONA EXCLUSIVAMENTE PARA OS ARQUIVOS DE CSV INPE===============================

In [ ]:
# =============================FUNÇAO QUE CALCULA O VENTO ========================== 
# ====================================================== =================== =====

def calcula_vento(arquivo):
    arquivo = pd.DataFrame(arquivo)
                         
    arquivo = arquivo.astype(float)
       
    lon = (arquivo['lon'])
    lat = (arquivo['lat'])
    m = (arquivo['spd'])
    u = m * np.sin((360 - arquivo['dir']) * np.pi/180)
    v = -m * np.cos((360- arquivo['dir']) * np.pi/180)
    u_norm = u / np.sqrt(u ** 2.0 + v ** 2.0)
    v_norm = v / np.sqrt(u ** 2.0 + v ** 2.0)
    
    return lon, lat, u_norm, v_norm
# ================================================================================================================
# EXEMPLO DE COMO O DADO DEVERIA ABRIR SEM PRECISAR CHAMAR A FUNÇÃO 
# lon = (dados_inpe_100_200['lon'])
# lat = (dados_inpe_100_200['lat'])
# m = dados_inpe_100_200['spd']

# u = m * np.sin((360 - dados_inpe_100_200['dir']) * np.pi/180)
# v = -m * np.cos((360- dados_inpe_100_200['dir']) * np.pi/180) 

# u_norm = u / np.sqrt(u ** 2.0 + v ** 2.0)
# v_norm = v / np.sqrt(u ** 2.0 + v ** 2.0)
# ================================================================================================================
# #============= OBS
# Não consegui resolver o problema para enviar qlqr arquivo 
# long,lati, u_norm, v_norm = fpl.calcula_vento(dados_inpe_100_200,'lat', 'lon', 'spd', 'dir' )
# neste exemplo quando retorna, acaba acontecendo um erro de conversao em str e int , nao descobrir o pq, mas acontece
# quando mando o nome dos parametros 'LAT', LON OU ALGO ASSIM
# ================================================================================================================
# ========= EXPLICAÇÃO DA FUNCAO 
# chame a função mandando um dataframe já aberto, logo apos dentro da função ele reabre o dataframe
# pega as variaveis, direcao do vento, velocidade do vento, lat e lon 
# clacula o vento e retorna lat,lon, u_norm e v_norm

# ================================================================================================================
#==============================FUNCIONA EXCLUSIVAMENTE PARA OS ARQUIVOS DE CSV INPE===============================

In [1]:

#-----------------------------------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------------------------------
def reprojetando_imagens( arquivo_acht, caminho_acht, var, my_extent):
    # CRIANDO PASTAS DE DADOS 
    #input = "DADOS/GOES ACHT/"; os.makedirs(input, exist_ok=True)
    # output = caminho_acht + "Output"; os.makedirs(output, exist_ok=True)
    output = caminho_acht; os.makedirs(output, exist_ok=True)

    # PEGANDO A EXTENSAO DEFINIDA 
    extent = my_extent
    file_name = arquivo_acht
    #-----------------------------------------------------------------------------------------------------------
    # Variable
    var =  var

    # Open the file
    #img = gdal.Open(f'NETCDF:{input}/{file_name}:' + var)
    img = gdal.Open(f'NETCDF:{file_name}:' + var)

    # Data Quality Flag (DQF)
    dqf = gdal.Open(f'NETCDF:{file_name}:DQF')

    # Read the header metadata
    metadata = img.GetMetadata()
    scale = float(metadata.get(var + '#scale_factor'))
    offset = float(metadata.get(var + '#add_offset'))
    undef = float(metadata.get(var + '#_FillValue'))
    dtime = metadata.get('NC_GLOBAL#time_coverage_start')
    
    # Load the data
    ds = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize).astype(float)
    ds_dqf = dqf.ReadAsArray(0, 0, dqf.RasterXSize, dqf.RasterYSize).astype(float)

    # Apply the scale, offset and convert to celsius
    ds = (ds * scale + offset)

    # Apply NaN's where the quality flag is greater than 1
    ds[ds_dqf > 1] = np.nan

    # Reproject the file
    nome_imagem = file_name.split('/')[-1]
    filename_ds = f'{output}/{nome_imagem}_ret.nc'
    reproject(filename_ds, img, ds, extent, undef) # salvando o arquivo no output 
    print('Pronto a reprojeçao do arquivo, salvo em \n ', filename_ds)
    return filename_ds
#-----------------------------------------------------------------------------------------------------------


In [2]:
# ===================================REPROJETAR A IMAGEM DO PRODUTO GOES 16 L2 - ==============================
# LINK ---> https://www.youtube.com/watch?v=ESlTKAGRgbY&t=1s  a partir das 4 horas de video
# LINK - >  SCRIPT 14 E 15 Da pasta do curso do GNC 2021 
# ================================================================================================================
# Function to reproject the data
def reproject(file_name, ncfile, array, extent, undef):

    # Read the original file projection and configure the output projection
    source_prj = osr.SpatialReference()
    source_prj.ImportFromProj4(ncfile.GetProjectionRef())

    target_prj = osr.SpatialReference()
    target_prj.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
   
    # Reproject the data
    GeoT = ncfile.GetGeoTransform()
    driver = gdal.GetDriverByName('MEM')
    raw = driver.Create('raw', array.shape[0], array.shape[1], 1, gdal.GDT_Float32)
    raw.SetGeoTransform(GeoT)
    raw.GetRasterBand(1).WriteArray(array)

    # Define the parameters of the output file  
    kwargs = {'format': 'netCDF', \
            'srcSRS': source_prj, \
            'dstSRS': target_prj, \
            'outputBounds': (extent[0], extent[3], extent[2], extent[1]), \
            'outputBoundsSRS': target_prj, \
            'outputType': gdal.GDT_Float32, \
            'srcNodata': undef, \
            'dstNodata': 'nan', \
            'resampleAlg': gdal.GRA_NearestNeighbour}

    # Write the reprojected file on disk
    gdal.Warp(file_name, raw, **kwargs)

# ================================================================================================================
# explicação da função ainda nao fiz ..
# ================================================================================================================

In [1]:
## ========================================== PARA OBTER O VALOR NO PONTO ESPECIFICO ===========================
##============= E RECORTAR AO REDOR DO PONTO =====================================
#
## ======================================================================================================
#def criando_recorte(filename_ds, lon, lat):
## Caminho para o arquivo NetCDF
#    file_path = filename_ds
#    
#    # latitude_desejada = float(dados_inpe_100_200['lat'].values[1])   # Substitua pela latitude desejada
#    # longitude_desejada = float(dados_inpe_100_200['lon'].values[1])  # Substitua pela longitude desejada
#    latitude_desejada = float(lat)   # Substitua pela latitude desejada
#    longitude_desejada = float(lon)  # Substitua pela longitude desejada
#
#    # Carregue o arquivo NetCDF usando xarray
#    dataset = xr.open_dataset(file_path)
#
#    # Encontre os índices mais próximos para a latitude e longitude desejadas
#    lon_index = abs(dataset['lon'] - longitude_desejada).argmin().item()
#    lat_index = abs(dataset['lat'] - latitude_desejada).argmin().item()
#    
#
#    # Extraia o ponto específico
#    try:
#        dados_ponto = dataset['Band1'][lon_index, lat_index].item()
#        #dados_ponto = dataset['Band1'][longitude_desejada,latitude_desejada].item()
#    except IndexError:
#        #print(f"Erro ao acessar os dados no ponto lon {lon_index} lat {lat_index}")
#        dataset.close()
#        return None, None
#
##     dados_ponto = dataset['Band1'][lon_index, lat_index].item()
#    #print("Valor na posição (lat={}, lon={}): {}".format(latitude_desejada, longitude_desejada,dados_ponto))
#        
#     # Verifique as dimensões do arquivo
#    max_lon_index = dataset.dims['lon'] - 1
#    max_lat_index = dataset.dims['lat'] - 1
#
#    # Ajuste os índices para garantir que o recorte não ultrapasse as dimensões do arquivo
#    lon_start = max(0, lon_index - 9)
#    lon_end = min(max_lon_index, lon_index + 10)
#    lat_start = max(0, lat_index - 9)
#    lat_end = min(max_lat_index, lat_index + 10)
#
#     # Verifique se os índices estão dentro dos limites
#    if lon_start > max_lon_index or lon_end >= dataset.sizes['lon'] or \
#            lat_start > max_lat_index or lat_end >= dataset.sizes['lat']:
#            dataset.close()
#            return None, None
#    
#        # Recorte a região 19x19 pixels ao redor do ponto desejado
#    recorte = dataset['Band1'][lon_start:lon_end, lat_start:lat_end]
#
##     recorte = dataset['Band1'][lon_index - 9:lon_index + 10, 
##                                            lat_index - 9 : lat_index + 10 ]
#
#    #print("Valor para o ponto de latitude {} e longitude {}: {}".format(latitude_desejada, longitude_desejada, dados_ponto))
##     print('forma do recorte: ', recorte.shape)
#
#    # Feche o arquivo NetCDF
#    dataset.close()
#    return recorte, dados_ponto
## fim
#
## ====================================================================================================== VERSAO 2 -========================

def criando_recorte(filename_ds, lon, lat): 
# Caminho para o arquivo NetCDF
    file_path = filename_ds
    
    latitude_desejada = float(lat)   # Substitua pela latitude desejada
    longitude_desejada = float(lon)  # Substitua pela longitude desejada

    # Carregue o arquivo NetCDF usando xarray
    dataset = xr.open_dataset(file_path)

    # Encontre os índices mais próximos para a latitude e longitude desejadas
    lon_index = abs(dataset['lon'] - longitude_desejada).argmin().item()
    lat_index = abs(dataset['lat'] - latitude_desejada).argmin().item()
    
    valor_pixel = dataset['Band1'][lon_index, lat_index].item()

 #     dados_ponto = dataset['Band1'][lon_index, lat_index].item()
    #print("Valor na posição (lat={}, lon={}): {}".format(latitude_desejada, longitude_desejada,dados_ponto))
        
     # Verifique as dimensões do arquivo
    max_lon_index = dataset.dims['lon'] - 1
    max_lat_index = dataset.dims['lat'] - 1

    # Ajuste os índices para garantir que o recorte não ultrapasse as dimensões do arquivo
    lon_start = max(0, lon_index - 9)
    lon_end = min(max_lon_index, lon_index + 10)
    lat_start = max(0, lat_index - 9)
    lat_end = min(max_lat_index, lat_index + 10)

     # Verifique se os índices estão dentro dos limites
    if lon_start > max_lon_index or lon_end >= dataset.sizes['lon'] or \
            lat_start > max_lat_index or lat_end >= dataset.sizes['lat']:
            dataset.close()
            return None, None
    
        # Recorte a região 19x19 pixels ao redor do ponto desejado
    recorte = dataset['Band1'][lon_start:lon_end, lat_start:lat_end]

    # Feche o arquivo NetCDF
    dataset.close()
    return recorte, valor_pixel
# fim
# ==================================================================================

In [ ]:
# ================ FUNCAO PARA ARRUMAR OS 9 MIN A MAIS DO ARQUIVO
from datetime import timedelta

def arruma_9(data):
    delta = timedelta(minutes = 9)
    data_arrumada =data -delta 
#     print(data_arrumada )
    return data_arrumada


# ======

In [1]:
# MOSTRANDO PIXEL A PIXEL PARA VERIFICAR O PIXEL CENTRAL E SALVANDO EM UM VETOR PARA CALCULAR AS MEDIAS 
# ======================================================================================================
# MOSTRANDO PIXEL A PIXEL PARA VERIFICAR O PIXEL CENTRAL E SALVANDO EM UM VETOR PARA CALCULAR AS MEDIAS 
# ======================================================================================================

def vetor_dados_recorte(recorte):
    
    vetor_aux = []
    linha = recorte.shape[0] 
    coluna = recorte.shape[1]
    if linha * coluna == 361:
#         print(linha, coluna)
        for i in range(linha):  # Itera sobre as linhas (latitude)
            for j in range(coluna):  # Itera sobre as colunas (longitude)
                valor_pixel = recorte[j,i ].values  # Acessa o valor do pixel
                vetor_aux.append(valor_pixel)
                # Faça o que quiser com o valor do pixel, por exemplo, imprimir
                #print("Valor do pixel na posição ({}, {}): {}".format(j, i, valor_pixel))
        #print(" tamanho do vetor ", len(vetor_aux))
        #vetor_aux = float(vetor_aux)
        return vetor_aux
    else:
        return None

# ==========================================================================================================        


In [ ]:
#===================== CRIANDO FUNCAO PARA CALCULAR  AS PORCENTAGENS ==================================
#=====================================================================================
#vetor_aux = vetor_dados_recorte(recorte)
# ====== PEGANDO OS % MAIS FRIO E MAIS QUENTES 
# a variavel valor é quem recebe a porcentaem a ser calculada
def porcentagens(vetor_aux, valor):
    vet_ord = np.sort(vetor_aux)
    vet_ord_inv = vet_ord[::-1]

    n = int(valor * len(vet_ord))
    vet_frio =  vet_ord[:n]
    vet_quente = vet_ord_inv[:n]

# =============================================================================================
    quente = np.nanmean(vet_frio)
    quente = quente.round(2)
    frio =  np.nanmean(vet_quente)
    frio = quente.round(2)
    media_geral = np.nanmean(vet_ord)
    media_geral = media_geral.round(2)
    print(" {} mais frios \n  {} todos " .format(quente, media_geral))
    return quente,frio, media_geral

# np.nanmean(vet_30_quente)


# ======================================================================================================
# EXPLICAÇÕES 



# ======================================================================================================

In [ ]:
# funcao desvio padrao
def calc_dev_padrao(vetor_aux):
    vetor_aux = np.nanmean(vetor_aux)
    desv = np.std(vetor_aux)
    return desv

In [1]:
# ======================================FUNÇÃO PARA ARMAZENAR OS ARQUIVOS EM UM CSV =============================
# ======================================================================================================
# dados = [lat, lon, valor_pixel_no_ponto, quente,frio, media_todos,
#          float(dados_inpe_100_200['temp'].values[4]),
#          float(dados_inpe_100_200['press'].values[4])]
# colunas = [ 'lat', 'lon', 'temp_na_lat_lon_exata', 'media_quente',
#            'media_frio', 'media_19x19', 'Temp_AMV_INPE', 'PRESS_AMV_INPE' ] 

# def armazena_dados( caminho_arquivo, lista_arquivos, dados, colunas):
#     output = caminho_arquivo + "Output_txt"; os.makedirs(output, exist_ok=True)
#     file_name = lista_arquivos
#     nome_imagem = file_name.split('/')[-1]
#     nome_arquivo = f'{output}/{nome_imagem}_ret.csv'
    
#     if os.path.isfile(nome_arquivo):
#         # Se o arquivo existe, abra-o e salve os dados
#         df = pd.read_csv(nome_arquivo)
#         novo_dados = pd.DataFrame([dados], columns=colunas )
#         df = df.append(novo_dados, ignore_index=True)
#         df.to_csv(nome_arquivo, index=False ) # , sep = ','
#         print(f'Dados adicionados ao arquivo existente {nome_imagem}')
#     else:
#         # Se o arquivo não existe, crie um novo arquivo com as colunas e salve os dados
#         df = pd.DataFrame([dados], columns=colunas)
#         df.to_csv(nome_arquivo, index=False, sep = ',')
#         print(f'Novo arquivo criado e dados armazenados.{nome_imagem}')

# # armazena_dados( caminho_acht, arquivo_acht[1], dados, colunas)

# ==========================================================================================================

import numpy as np
import pandas as pd
def armazena_dados( caminho_arquivo, lista_arquivos, dados, colunas):
    output = caminho_arquivo + "Output_txt"; os.makedirs(output, exist_ok=True)
    file_name = lista_arquivos
    nome_imagem = file_name.split('/')[-1]
    nome_arquivo = f'{output}/{nome_imagem}_ret.csv'
    
    if os.path.isfile(nome_arquivo):
        # Se o arquivo existe, abra-o e salve os dados
        df = pd.read_csv(nome_arquivo)
        novo_dados = pd.DataFrame([dados], columns=colunas )
#         df = df.append(novo_dados, ignore_index=True)
        
        df = pd.concat([df, novo_dados], ignore_index=True)

#         df = df.append(pd.DataFrame([dados], columns=colunas), ignore_index=True)
        df.to_csv(nome_arquivo, index=False ) # , sep = ','
        print(f'Dados adicionados ao arquivo existente {nome_imagem}')
    else:
        # Se o arquivo não existe, crie um novo arquivo com as colunas e salve os dados
        df = pd.DataFrame([dados], columns=colunas)
        df.to_csv(nome_arquivo, index=False, sep = ',')
        print(f'Novo arquivo criado e dados armazenados.{nome_imagem}')

# armazena_dados( caminho_acht, arquivo_acht[1], dados, colunas)








# ======================================================================================================
# EXPLICAÇOES



# ======================================================================================================

In [ ]:
# ====================================================================================
# ====================================================================================

import datetime as dt

def converte_dia_juliano(dia_juliano, ano):
    dia_juliano= int(dia_juliano)
    ano = int(ano)
    
    # Calcula a data a partir do dia juliano e ano
    data_inicio_ano = dt.datetime(ano, 1, 1)
    data_dia_juliano = data_inicio_ano + dt.timedelta(days = dia_juliano - 1)

#     return data_dia_juliano.strftime('%Y-%m-%d')
    return data_dia_juliano
# ====================================================================================




# ====================================================================================

In [ ]:
# ====================================================================================
# FUNÃO PARA CONVERTAR UMA PARTE DO ARQUIVO TEXTO DO NC PARA HORA
# FUNCAO QUEBRA GALHO

def convertendo_em_hora_mes_dia(arquivo):
    dados = arquivo
    separando = dados.split('/')[-1]
    sep_data_hora = separando.split('_')[-3]
    ano = sep_data_hora[1:5]
    dj = sep_data_hora[5:8]
    hora = sep_data_hora[8:12]
    data_arquivo = converte_dia_juliano(dj, ano)
    data_e_hora = data_arquivo.strftime('%Y%m%d') + hora
    data_e_hora  = dt.datetime.strptime(data_e_hora, '%Y%m%d%H%M')
    return data_e_hora

import datetime as dt

def converte_dia_juliano(dia_juliano, ano):
    dia_juliano= int(dia_juliano)
    ano = int(ano)
    
    # Calcula a data a partir do dia juliano e ano
    data_inicio_ano = dt.datetime(ano, 1, 1)
    data_dia_juliano = data_inicio_ano + dt.timedelta(days = dia_juliano - 1)

#     return data_dia_juliano.strftime('%Y-%m-%d')
    return data_dia_juliano
# 

In [1]:
#==== =============================== PLOTANDO RECORTE ========================================================

def plotando_recorte(lat, lon, recorte):
    # Coordenadas do ponto central do recorte
    latitude_central = float(lat)  # Substitua pela latitude central do recorte
    longitude_central = float(lon)  # Substitua pela longitude central do recorte
    tam_grade = 10
    # Criando arrays de latitude e longitude para o recorte
    latitudes = np.linspace(latitude_central - int(tam_grade), 
                            latitude_central + int(tam_grade), num=recorte.shape[0])
    longitudes = np.linspace(longitude_central - int(tam_grade), 
                            longitude_central + int(tam_grade), num=recorte.shape[1])

    data = plt.imshow(recorte.values, cmap='Greys_r',
              extent=[longitudes[0], longitudes[-1], latitudes[-1], latitudes[0]])  # Plotando a imagem diretamente

    # #=============
    # long,lati, u_norm, v_norm = fpl.calcula_vento(dados_inpe_100_200) #,'lat', 'lon', 'spd', 'dir' )

    # plt.quiver(long, lati, v_norm , u_norm,   scale= 60,  
    #                     pivot = "tip", alpha=1,
    #                     #width = 0.005  ,
    #                     color =  'pink'  #'lightblue' 
    # #             
    #                       )

    plt.plot(lon, lat, 'ro', markersize=3)


    #plt.scatter(recorte.shape,recorte.shape,recorte.values)
    plt.colorbar(data )  # Adiciona uma barra de cores para a escala
    plt.title('Recorte 19x19 pixels', loc = 'left')
    plt.title(str(latitude_central) + str(longitude_central) ,
              loc = 'right', fontsize = 10)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()
    
# ============



In [ ]:
# ==========================GERANDO GIF =============================================================
# ==========================GERANDO GIF=======================================================
# ==================================================================================================
# import shutil
# import imageio
# import matplotlib.pyplot as plt
# from IPython.display import display, Image


# def gerando_gif(caminho_imagem, saida, nome_arquivo):
    # 
    # imagens = fpl.retorna_lista(caminho_imagem)
# 
    ## Crie o GIF
    # fps = 5
    # with imageio.get_writer('animacao.gif', mode='I', duration = 1.0/ fps) as writer:
        # for imagem in imagens:
            # imagem = imageio.imread(imagem)
            ##writer.append_data(imagem)
#     with open('animacao.gif', 'rb') as file:
#         display(Image(file.read()))

# Para salvar o GIF em um local específico

#    shutil.move('animacao.gif', f'{saida}{nome_arquivo}.gif')

# =====================================================================
# =====================================================================

In [ ]:
# ---------------------criando funcao de definir altura ---------- NUNCA USEI =======
def calcular_altura(pressao, temperatura):
    #DEFININDO CONSTANTES 
    R = 8.314 #J/(mol·K)),
    P0 = 1000 #PRESSAO INICIAL
    P = pressao
    T = temperatura  # kelvin
    g = 9.81 #m/s2
  
    h = (R*T/g) * np.log(P0/P)
    #h= pd.DataFrame(h)
    return h
# ----------------------------------------------------------------------------
# -- DESCRICAO----
#  chame a funcao e  mande uma pressao em hpa e temperatura em kelvin
# 
# 

In [ ]:
# # ================================== FUNCAO REMAP PARA OS PRODUTOS DO GOES L2   =====================================
# # LINK DROPBOX ---> https://www.dropbox.com/s/r5q6e05i1sgw5br/remap.py?dl=0
# # LINK DROPBOX 2 ---> https://www.dropbox.com/s/2zylbwfjfkx9a7i/GNC-A%20Blog%20-%20GOES-R-Level-2-Products.py?dl=0
# # LINK GNC EXPLICANDO --> https://geonetcast.wordpress.com/2018/06/28/goes-r-level-2-products-a-python-script/
# # 
# # =========================================================================
# #from netCDF4 import Dataset
# # import numpy as np
# # from osgeo import osr
# # from osgeo import gdal
# # import time as t

# # Define KM_PER_DEGREE
# KM_PER_DEGREE = 111.32

# # GOES-16 Spatial Reference System
# sourcePrj = osr.SpatialReference()
# #sourcePrj.ImportFromProj4('+proj=geos +h=35786023.0 +a=6378137.0 +b=6356752.31414 +f=0.00335281068119356027489803406172 +lat_0=0.0 +lon_0=-75 +sweep=x +no_defs')
# sourcePrj.ImportFromProj4('+proj=geos +h=35786000 +a=6378140 +b=6356750 +lon_0=-75 +sweep=x')

# # Lat/lon WSG84 Spatial Reference System
# targetPrj = osr.SpatialReference()
# #targetPrj.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
# targetPrj.ImportFromProj4('+proj=latlong +datum=WGS84')

# def exportImage(image,path):
#     driver = gdal.GetDriverByName('netCDF')
#     return driver.CreateCopy(path,image,0)

# def getGeoT(extent, nlines, ncols):
#     # Compute resolution based on data dimension
#     resx = (extent[2] - extent[0]) / ncols
#     resy = (extent[3] - extent[1]) / nlines
#     return [extent[0], resx, 0, extent[3] , 0, -resy]

# def getScaleOffset(path, variable):
#     nc = Dataset(path, mode='r')
    
#     if (variable == "BCM") or (variable == "Phase") or (variable == "Smoke") or (variable == "Dust") or (variable == "Mask") or (variable == "Power"): 
#         scale = 1
#         offset = 0     
#     else:
#         scale = nc.variables[variable].scale_factor
#         offset = nc.variables[variable].add_offset
#     #scale = 0
#     #offset = 0
#     nc.close()
#     return scale, offset
    
# def remap(path, variable, extent, resolution, x1, y1, x2, y2):
    
#     scale = 1
#     offset = 0
    
#     # GOES-16 Extent (satellite projection) [llx, lly, urx, ury]
#     GOES16_EXTENT = [x1, y1, x2, y2]
    
#     # Setup NetCDF driver
#     gdal.SetConfigOption('GDAL_NETCDF_BOTTOMUP', 'NO')
        
#     if not (variable == "DQF"):              
#         # Read scale/offset from file
#         scale, offset = getScaleOffset(path, variable) 
#         #print(scale)
#         #print(offset)
      
#     connectionInfo = 'HDF5:\"' + path + '\"://' + variable
#     #print(connectionInfo)
#     #connectionInfo = 'HDF5:"E:\VLAB\Python\GOES-16 Samples - L2\OR_ABI-L2-CMIPF-M3C13_G16_s20180571300407_e20180571311185_c20180571311263.nc"://CMI'
#     raw = gdal.Open(connectionInfo)
       
#     #print("Connection Info:")
#     #print(connectionInfo)            
    
#     # Setup projection and geo-transformation
#     raw.SetProjection(sourcePrj.ExportToWkt())
#     #raw.SetGeoTransform(getGeoT(GOES16_EXTENT, raw.RasterYSize, raw.RasterXSize))
#     raw.SetGeoTransform(getGeoT(GOES16_EXTENT, raw.RasterYSize, raw.RasterXSize))  
  
#     #print (KM_PER_DEGREE)
#     # Compute grid dimension
#     sizex = int(((extent[2] - extent[0]) * KM_PER_DEGREE) / resolution)
#     sizey = int(((extent[3] - extent[1]) * KM_PER_DEGREE) / resolution)
    
#     # Get memory driver
#     memDriver = gdal.GetDriverByName('MEM')
   
#     # Create grid
#     grid = memDriver.Create('grid', sizex, sizey, 1, gdal.GDT_Float32)
        
#     # Setup projection and geo-transformation
#     grid.SetProjection(targetPrj.ExportToWkt())
#     grid.SetGeoTransform(getGeoT(extent, grid.RasterYSize, grid.RasterXSize))

#     # Perform the projection/resampling 

#     print ('Remapeando... ', path)
        
#     start = t.time()
    
#     gdal.ReprojectImage(raw, grid, sourcePrj.ExportToWkt(), targetPrj.ExportToWkt(), 
#                         gdal.GRA_NearestNeighbour, options=['NUM_THREADS=ALL_CPUS']) 
    
#     print ('Pronto!! Tempo:', t.time() - start, 'segundos')
    
#     # Read grid data
#     array = grid.ReadAsArray()
    
#     # Mask fill values (i.e. invalid values)
#     np.ma.masked_where(array, array == -1, False)
    
#     array = array.astype(np.uint16)  
       
#     # Apply scale and offset
#     array = array * scale + offset
#     #np.ma.masked_where(array, array == 100, False)
#     #print(array)
    
#     #grid.GetRasterBand(1).SetNoDataValue(-1)
#     grid.GetRasterBand(1).WriteArray(array)

#     # fechando arquivo
#     raw = None
        
#     return grid


# # =========================================================================